## Some code to align a shapefile

Shift all features in the shapefile such that the top-left point of their bounding box aligns to the mastergrid or something else.

Might need to do this if we've got a bounding box or series of them that someone wants to use to clip mastergrid aligned rasters with predictable clean pixel match-ups.

In [6]:
from raster_utilities.utils.geotransform_calcs import SnapAndAlignGeoTransform, SnapTypes

In [1]:
# get information about the shapefile 
!ogrinfo -al -so "F:\Shared drives\ML PR modelling\Data\PR_Locations\PR_Points\PR_10k_Boxes_Ungrouped.shp"

INFO: Open of `F:\Shared drives\ML PR modelling\Data\PR_Locations\PR_Points\PR_10k_Boxes_Ungrouped.shp'
      using driver `ESRI Shapefile' successful.

Layer name: PR_10k_Boxes_Ungrouped
Metadata:
  DBF_DATE_LAST_UPDATE=2019-04-04
Geometry: Polygon
Feature Count: 43877
Extent: (-3504692.107080, -3027616.184459) - (3950520.365058, 3898075.855706)
Layer SRS WKT:
PROJCS["Africa_Sinusoidal",
    GEOGCS["GCS_WGS_1984",
        DATUM["WGS_1984",
            SPHEROID["WGS_84",6378137.0,298.257223563]],
        PRIMEM["Greenwich",0.0],
        UNIT["Degree",0.0174532925199433]],
    PROJECTION["Sinusoidal"],
    PARAMETER["False_Easting",0.0],
    PARAMETER["False_Northing",0.0],
    PARAMETER["longitude_of_center",15.0],
    UNIT["Meter",1.0],
    AUTHORITY["Esri","102011"]]
ident: String (16.0)
Npositive: Integer64 (10.0)
Nexamined: Integer64 (10.0)
PfPr: Real (31.15)
year: Integer (6.0)
latitude: Real (31.15)
longitude: Real (31.15)


In [19]:
# get the bounding box of the features 
# https://pcjericks.github.io/py-gdalogr-cookbook/vector_layers.html
inPath = "F:\Shared drives\ML PR modelling\Data\PR_Locations\PR_Points\PR_10k_Boxes_Ungrouped.shp"
outPath = inPath.replace("Ungrouped", "Ungrouped_Aligned30m")
from osgeo import ogr
d = ogr.Open(inPath)
l = d.GetLayerByIndex(0)
x_min, x_max, y_min, y_max = l.GetExtent()

'PR_10k_Boxes_Ungrouped'

In [3]:
# make a raster geotransform-like object for this extent, given a resolution to align to
# here we have a shapefile in projected meters, not a lat/lon system, so say res of 30m
alignToMGResolution = 30.0
fakeGT = (x_min, alignToMGResolution, 0.0, y_max, 0.0, -alignToMGResolution)

In [7]:
# use the MG snapping code to re-position it
snappedGT = SnapAndAlignGeoTransform(inGT=fakeGT, fixResolution=True, snapType = SnapTypes.NEAREST)

INFO    : | Cellsize already ok, not altering
INFO    : | Snapped origin point from (-3504692.107079846, 3898075.8557064408) (x,y) to (-3504690.0, 3898080.0) (x,y)


In [9]:
# and see what translation this corresponds to
translation_X = snappedGT[0] - fakeGT[0]
translation_Y = snappedGT[3] - fakeGT[3]
(translation_X, translation_Y)

(2.10707984585315, 4.1442935592494905)

Now to actually translate the features we use a bit of a hack, taking advantage of the fact that ogr2ogr lets us run spatial SQL against a shapefile. Idea from:
https://gis.stackexchange.com/a/73162/10742

In [21]:
ogrCommand = 'ogr2ogr "{}" "{}" -dialect sqlite -sql "SELECT ShiftCoords(geometry,{},{}) FROM {}"'.format(
    r"C:\temp\test.shp", inPath, translation_X, translation_Y, l.GetName())

In [22]:
!{ogrCommand}

The extent of the shapefile should now align with a mastergrid of the specified resolution

Here's a more brute force way of moving the features (inplace this time)

In [ ]:
driver = ogr.GetDriverByName("ESRI Shapefile")
dataSource = driver.Open(inPath,1)
layer = dataSource.GetLayer(0)
for feature in layer:
    get_poly = feature.GetGeometryRef()
    get_ring = get_poly.GetGeometryRef(0)
    points   = get_ring.GetPointCount()
    set_ring = ogr.Geometry(ogr.wkbLinearRing)
    for p in xrange(points):
        x,y,z = get_ring.GetPoint(p)
        x += dx
        y += dy
        z += dz
        set_ring.AddPoint(x,y)
        print x,y,z
set_poly = ogr.Geometry(ogr.wkbPolygon)
set_poly.AddGeometry(set_ring)

feature.SetGeometry(set_poly)
layer.SetFeature(feature)

del layer
del feature
del dataSource   